http://hadron.physics.fsu.edu/~skpark/document/ROOT/roofit-intro-roostats-v12a.pdf

In [ ]:
using DataFrames
using Random
using PyPlot

function gaus(x, μ, σ, n)
    n*exp(-(x-μ)^2/2/σ^2)/sqrt(2π*σ^2)
end

a = randn(600).*2.3 .+ 23.0
b = randn(500).*3.2 .+ 30.1

c = vcat(a, b)

#plt.hist(a, bins=collect(1:1:50), alpha=0.25)
#plt.hist(b, bins=collect(1:1:50), alpha=0.25)
plt.hist(c, bins=collect(1:1:50))

x = collect(1:0.1:50)
y1 = gaus.(x, 23.0, 2.3, 600)
y2 = gaus.(x, 30.1, 3.2, 500)
#plt.plot(x, y1)
plt.plot(x, y2+y1)

plt.show()

In [ ]:
using BenchmarkTools
mutable struct Variable
    name::String
    init::Float64
end

A = Variable("Aye", 1.0)
B = Variable("Bee", 2.0)
C = Variable("Sea", 3.0)

mutable struct pdf
    func::String
    params::Array{Variable}
    function pdf(f::String, p::Variable...)
        new(f, [p...])
    end
end

function one(a::Float64, b::Float64)
    a+b
end

function two(c::Float64, b::Float64)
    b+c
end

function three(c::Float64, a::Float64)
    c+2*a
end

p1 = pdf("one", A, B)
p2 = pdf("two", C, B)
p3 = pdf("three", C, A)

struct mod
    objective::Function
    numparams::Int64
    # Things here
    function mod(pf::Array{pdf})
        var_list = []
        for p in pf
            for v in p.params
                push!(var_list, v)
            end
        end
        vv = unique(var_list)
        nparams = length(vv)
        npdf = length(pf)
        matches = []
        for (idx,p) in enumerate(pf)
            match = []
            for v in p.params
                loc = findlast(x->x==v, vv)
                push!(match, loc)
            end
            push!(matches, match)
        end
        seval = ""
        z = collect(1:1:nparams)
        for (idx, p) in enumerate(pf)
            a = matches[idx]
            name = pf[idx].func
            seval *= "$name("
            for i in a
                seval *= "x[$i],"
            end
            seval *= ") +"
        end
        seval = seval[1:end-1]
        #seval = "x.^2"
        @show seval
        ff = eval(Meta.parse("x->"*seval))
        new(ff, nparams)
    end
end

m = mod([p1, p2, p3])
@show m.objective([1.0, 2.0, 3.0])

blah = m.objective
@benchmark m.objective([2.0, 2.0, 2.0])

In [ ]:
push!(LOAD_PATH, "../src/")
using Batman
using NLopt

### our goal here is to use the new interface to reproduce the fish code

s = Variable("Signal")
μs = Variable("Signal Mean")
σs = Variable("Signal Diff")

b = Variable("Background")
μb = Variable("Background Mean")
σb = Variable("Background Diff")

function myshape(x, μ, σ, n)
    n*exp(-(x-μ).^2/2/σ^2)/sqrt(2πσ^2)
end

# s*myshape(x, mu_s...) + b*myshape(x, mu_b...)
data = Variable("Data")

#@addfunction logpoisson(s,b,n)=s+b-n*log(s+b)+n*log(n)-n
#@addfunction logpoisson(s,b,n) = begin
#    s+b-n*log(s+b)+n*log(n)-n
#end
@addfunction lognormalconstraint(x,μ,σ) = (x-μ)^2/2/σ

objlist = Array{NLogPDF}(undef,0)

push!(objlist, NLogPDF("logpoisson", s, b, data))
push!(objlist, NLogPDF("lognormalconstraint", s, μs, σs ))
push!(objlist, NLogPDF("lognormalconstraint", b, μb, σb ))

@show objlist

nll = NLogLikelihood(objlist)
@show nll.numparams

nll.objective([3,2,6,2.0,3.0,2.0,3.0])

In [ ]:
using Random
using DataFrames
dx = randn(100)
dy = randn(100)

df = DataFrame(energy=dx, position=dy)



In [ ]:
struct mynum
    x::Float64
end

a = mynum(12.0)
b = mynum(13.0)

for op = (:+, :*, :/, :-)
    eval(quote
        Base.$op(a::mynum, b::mynum) = mynum( $op(a.x, b.x) )
    end)
end

In [ ]:
function fFromS(str)
    f = eval( Meta.parse("x -> " *str))
    return x -> Base.invokelatest(f, x)
end

one = "A + B"
two = "B + C"

mutable struct Variable
    name::String
    init::Float64
end

A = Variable("A", 1.0)
B = Variable("B", 2.0)
C = Variable("C", 3.0)

struct combo
    f::Function
    function combo(flist)
        
        

In [ ]:
function fone(a, b)
    R = 0
    R += one(a, b)
    R
end
@benchmark one([1.0, 1.0]...)

In [ ]:
struct k
    params::Array{Float64}
    function k(p...)
        new([p...])
    end
end

blah = k(1.32, 21.0, 14.0)
@show blah

In [ ]:
macro foo(pf::Array{Float64})
#    var_list = []
#     for p in pf
#         for v in p.params
#             push!(var_list, v)
#         end
#     end
    #show(pf)
end
k = [p1]

macro bar(x)
    show(x)
end

@foo(k)

In [ ]:
function f(a, b, c)
    function A(a, b)
        a+b
    end
    function B(b, c)
        b+c
    end
    A(a, b) + B(b, c)
end

function q(a, b)
    a+b
end
function l(b, c)
    b+c
end
function g(a, b, c)
    q(a, b) + l(b, c)
end

@benchmark g(1.0,2.0,3.0)

In [ ]:
using DataFrames

macro ndata( name, df )
    :( $(esc(name)) = $df )
end
@ndata morgan DataFrame(x=[1])

morgan